# Create & update keywords dictionary

- Create pools of responses for each item (ignoring the argument order)
- Update the pools as more data points are collected
- 

In [1]:
import numpy as np
import pandas as pd
import pickle

import collections

In [2]:
# transform pandas dataframe into pool of responses
def df2pool(df0, col_inum = "item_num", col_resp = "response", inum_list = list(range(1,61))):
    #df1 = df0.dropna(how = "any")
    
    # exclude uncertain transcriptions
    df1 = df0[[(type(f) != str) or ("uncertain" not in f) for f in df0["notes"]]]
    
    # excluded NOT_RECOGNIZED
    df = df1[~df1["response"].isin(["NOT_RECOGNIZED", "NEED_HELP"])]
    
    
    resp_pool = [list(df[df[col_inum] == i][col_resp]) for i in inum_list]
    pool_dict = dict(zip(inum_list, resp_pool))
    
    return pool_dict

In [3]:
# combine two pools
def combine_pools(pool1, pool2):
    pool_dict = dict()
    for k in pool1.keys():
        pool_dict[k] = pool1[k] + pool2[k]
    
    return pool_dict

In [4]:
def pool2keyword(pool):
    dictionary = dict()
    for k in pool.keys():
        resp_count = collections.Counter(pool[k])
        key_list = [r for r in resp_count.keys() if resp_count[r] > 1]
        dictionary[k] = key_list
    return dictionary

## Update the pool

In [5]:
# read the latest pool
response_pool = pickle.load( open( "/Users/masato/Box/cloze_experiments/automatic_transcription/keywords/pool_1110.p", "rb" ) )

In [6]:
new_df = pd.read_csv("/Users/masato/Box/cloze_experiments/filter/batch2/result_batch2.csv")
pool2 = df2pool(new_df)

In [7]:
new_pool = combine_pools(response_pool, pool2)

In [8]:
#pickle.dump(new_pool, open("/Users/masato/Box/cloze_experiments/automatic_transcription/keywords/pool_1129.p", "wb" ) )

## Output keyword list

In [9]:
keyword_list = pool2keyword(new_pool)

In [10]:
#pickle.dump(keyword_list, open("/Users/masato/Box/cloze_experiments/automatic_transcription/keywords/keywords_1129.p", "wb" ) )